## 正向传播、反向传播和计算图

只提供了模型的正向传播$（forward propagation）$的计算，即对输入计算模型输出，然后通过$autograd$模块来调用系统自动生成的$backward$函数计算梯度。基于反向传播$（back-propagation）$算法的自动求梯度极大简化了深度学习模型训练算法的实现。

下面将使用数学和计算图$（computational graph）$两个方式来描述正向传播和反向传播。具体来说，我们将以带$L_2$范数正则化的含单隐藏层的多层感知机为样例模型解释正向传播和反向传播。

**正向传播**

正向传播是指对神经网络沿着从输入层到输出层的顺序，依次计算并存储模型的中间变量（包括输出）。为简单起见，假设输出是一个特征为$\mathbf{x}\epsilon\mathbb{R}^d$的样本，且不考虑偏差项，那么中间变量
<center>
    $ \mathbf{z}=\mathbf{W}^{(1)}\mathbf{x} $
</center>
其中$ \mathbf{W}^{(1)}\epsilon\mathbb{R}^{h\times d} $是隐藏层的权重参数。把中间变量$ \mathbf{z}\epsilon\mathbb{R}^h $输入按元素运算的激活函数后，将得到向量长度为$h$的隐藏层变量
<center>
    $ \mathbf{h}=\phi(\mathbf{z}) $
</center>
隐藏层变量$ \mathbf{h} $也是一个中间变量。假设输出层参数$ \mathbf{W}\epsilon\mathbb{R}^{q\times h} $只有权重，可以得到向量长度为q的输出层变量
<center>
    $ \mathbf{o}=\mathbf{W}^{(2)}\mathbf{h} $
</center>
假设损失函数为$l$，且样本标签为$y$，可以计算出单个数据样本的损失项
<center>
    $ L=l(\mathbf{o},y) $
</center>
根据$L_2$范数正则化的定义，给定超参数$\lambda$，正则化项即
<center>
    $ s=\frac{\lambda}{2}(||\mathbf{W}^{(1)}||_F^2+||\mathbf{W}^{(2)}||_F^2) $
</center>
其中矩阵的$Frobenius$范数等价于将矩阵变平为向量后计算$L_2$范数。最终，模型在给定的数据样本上带正则化的损失为
<center>
    $ J=L+s $
</center>
我们将$J$称为有关给定数据样本的目标函数，并在以下的讨论中简称目标函数。

**反向传播**

反向传播指的是计算神经网络参数梯度的方法。总的来说，反向传播依据微积分中的链式法则，沿着从输出层到输入层的顺序，依次计算并存储目标函数有关神经网络各层的中间变量以及参数的梯度。对输入或输出$X,Y,Z$为任意形状张量的函数$Y=f(X)$和$Z=g(Y)$，通过链式法则，我们有
<center>
    $ \frac{\partial{Z}}{\partial{X}}=prod(\frac{\partial{Z}}{\partial{Y}}, \frac{\partial{Y}}{\partial{X}} )$
</center>
其中$prodprod$运算符将根据两个输入的形状，在必要的操作**（如转置和互换输入位置）后对两个输入做乘法**。

上面正向传播的参数为$ \mathbf{W}^{(1)} $和$ \mathbf{W}^{(2)} $，因此反向传播的目标是$ \frac{\partial{J}}{\partial{\mathbf{W}^{(1)}}}$和$ \frac{\partial{J}}{\partial{\mathbf{W}^{(2)}}}$。应用链式法则依次计算各中间变量和参数的梯度，计算次序和前向传播恰恰相反。首先，分别计算目标函数$ J=L+s $有关损失项和正则项的梯度

<center>
    $ \frac{\partial{J}}{\partial{L}}=1 $,   $ \frac{\partial{J}}{\partial{s}}=1 $
</center>

其次，依据链式法则计算目标函数有关输出层变量的梯度$ \frac{\partial{J}}{\partial{o}}\epsilon\mathbb{R}^q $

<center>
    $ \frac{\partial{J}}{\partial{o}}=prod(\frac{\partial{J}}{\partial{L}},\frac{\partial{L}}{\partial{o}})=\frac{\partial{L}}{\partial{o}} $
</center>

接下来，计算正则项有关两个参数的梯度

<center>
    $ \frac{\partial{s}}{\partial{\mathbf{W}^{(1)}}}=\lambda\mathbf{W}^{(1)} $, $ \frac{\partial{s}}{\partial{\mathbf{W}^{(2)}}}=\lambda\mathbf{W}^{(2)} $
</center>

现在，我们可以计算最靠近输出层的模型参数的梯度$\frac{\partial{J}}{\partial{\mathbf{W}^{(2)}}}\epsilon\mathbb{R}^{q\times h}$

<center>
    $ \frac{\partial{J}}{\partial{\mathbf{W}^{(2)}}}=prod(\frac{\partial{J}}{\partial{o}},\frac{\partial{o}}{\partial{\mathbf{W}^{(2)}}})+prod(\frac{\partial{J}}{\partial{s}},\frac{\partial{s}}{\partial{\mathbf{W}^{(2)}}})=\frac{\partial{J}}{\partial{o}}\mathbf{h}^T+\lambda\mathbf{W}^{(2)} $
</center>

沿着输出层向隐藏层继续反向传播，隐藏层变量的梯度$\frac{\partial{J}}{\partial{h}}\epsilon\mathbb{R}^h$可以这样计算

<center>
    $ \frac{\partial{J}}{\partial{h}}=prod(\frac{\partial{J}}{\partial{o}},\frac{\partial{o}}{\partial{h}})=\mathbf{W}^{(2)^T}\frac{\partial{J}}{\partial{o}} $
</center>

由于激活函数$\phi$是按元素运算的，中间变量$z$的梯度$\frac{\partial{J}}{\partial{z}}\epsilon\mathbb{R}^h$的计算需要使用按元素乘法符$\odot$

<center>
    $ \frac{\partial{J}}{\partial{z}}=prod(\frac{\partial{J}}{\partial{h}},\frac{\partial{h}}{\partial{z}})=\frac{\partial{J}}{\partial{h}}\odot\phi^{'}(z) $
</center>

最终可以得到最靠近输出层的模型参数的梯度$\frac{\partial{J}}{\partial{\mathbf{W}^{(1)}}}\epsilon\mathbb{R}^{h\times d}$

<center>
    $ \frac{\partial{J}}{\partial{\mathbf{W}^{(1)}}}=prod(\frac{\partial{J}}{\partial{z}},\frac{\partial{z}}{\partial{\mathbf{W}^{(1)}}})+prod(\frac{\partial{J}}{\partial{s}},\frac{\partial{s}}{\partial{\mathbf{W}^{(1)}}}) =\frac{\partial{J}}{\partial{z}}x^T+\lambda\mathbf{W}^{(1)}$
</center>

**训练深度学习模型**

在训练深度学习模型时，正向传播和反向传播之间相互依赖。

一方面，正向传播的计算可能依赖于**模型参数**的当前值，而这些模型参数是在反向传播的梯度计算后通过优化算法迭代的。

另一方面，反向传播的梯度计算可能依赖于各**变量**的当前值，而这些变量的当前值是通过正向传播计算得到的。

因此，在模型参数初始化完成后，我们交替地进行正向传播和反向传播，并根据反向传播计算的梯度迭代模型参数。**既然我们在反向传播中使用了正向传播中计算得到的中间变量来避免重复计算，那么这个复用也导致正向传播结束后不能立即释放中间变量内存。**这也是训练要比预测占用更多内存的一个重要原因。另外需要指出的是，这些中间变量的个数大体上与网络层数线性相关，每个变量的大小跟批量大小和输入个数也是线性相关的，它们是导致较深的神经网络使用较大批量训练时更容易超内存的主要原因。

+ 正向传播沿着从输入层到输出层的顺序，依次计算并存储神经网络的中间变量。
+ 反向传播沿着从输出层到输入层的顺序，依次计算并存储神经网络中间变量和参数的梯度。
+ 在训练深度学习模型时，正向传播和反向传播相互依赖。